In [595]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## import modules

In [596]:
import pandas as pd
import numpy as np
import seaborn as sns  #for statistical data visualization
import matplotlib.pyplot as plt #for basic graphs
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## load dataset

In [597]:
train =pd.read_csv('/kaggle/input/titanic/train.csv')
test =pd.read_csv('/kaggle/input/titanic/test.csv')
train.head()

In [598]:
train.describe()

In [599]:
train.info()

# Exploratory Data Analysis

In [600]:
## non survived vs survived
sns.countplot(train['Survived']);

In [601]:
sns.countplot(train['Pclass'])

In [602]:
sns.countplot(train['Survived'])

In [603]:
sns.countplot(train['Sex'])

In [604]:
sns.countplot(train['Embarked'])

In [605]:
sns.countplot(train['SibSp'])
#dominated by solo passengers and couples

In [606]:
sns.countplot(train['Parch'])

#### Numberical attributes

In [607]:
sns.distplot(train['Age'])

In [608]:
sns.distplot(train['Fare'])

In [609]:
class_fare = train.pivot_table(index='Pclass',values ='Fare')
class_fare.plot(kind='bar')
plt.title("average fare vs pclass")
plt.xlabel('Pclass')
plt.ylabel('Avg. Fare')
plt.xticks(rotation=0)
plt.show()

In [610]:
class_fare = train.pivot_table(index='Pclass',values ='Fare')
class_fare.plot(kind='bar')
plt.title("total fare vs pclass")
plt.xlabel('Pclass')
plt.ylabel('Total Fare')
plt.xticks(rotation=0)
plt.show()


In [611]:
sns.barplot(data=train,x='Pclass',y='Fare', hue='Survived')

In [612]:
sns.barplot(data=train,x='Survived',y='Fare', hue='Pclass')

### data preprocessing

In [613]:
train_len = len(train)
#combine
df=pd.concat([train,test],axis=0)
df=df.reset_index(drop=True)

In [614]:
df.head()

In [615]:
df.tail()

In [616]:
###find null val
df.isnull().sum()

In [617]:
#drop or delete the column
df = df.drop(columns=['Cabin'],axis=1)

In [618]:
#fill missing values using mean
df['Age']=df['Age'].fillna(df['Age'].mean())
df['Fare']=df['Fare'].fillna(df['Fare'].mean())

In [619]:
df['Embarked'].mode()[0]

In [620]:
df['Age']=df['Age'].fillna(df['Age'].mode()[0])

### log transform

In [621]:
sns.distplot(train['Fare'])

In [622]:
df['Fare']= np.log(df['Fare']+1)

In [623]:
sns.distplot(df['Fare'])

## correlation matrix

In [624]:
corr=df.corr()
plt.figure(figsize=(15,9))
sns.heatmap(corr,annot=True,cmap='coolwarm')

In [625]:
df.head()

In [626]:
## drop unecessary columns
df =df.drop(columns=['Name','Ticket'],axis=1)
df.head()

## Label Encoding

In [627]:
from sklearn.preprocessing import LabelEncoder
cols=['Sex','Embarked']
le= LabelEncoder()

for col in cols:
    df[col]= le.fit_transform(df[col])
df.head()

## Train-test split

In [628]:
train = df.iloc[:train_len,:]
test = df.iloc[train_len:,:]

In [629]:
test.head()

In [630]:
#input split
X=train.drop(columns=['PassengerId','Survived'],axis=1)
y=train['Survived']

In [631]:
X.head()

# Model training

In [632]:
from sklearn.model_selection import train_test_split , cross_val_score
def classify(model):
    x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
    model.fit(x_train,y_train)
    print('Accuracy:',model.score(x_test,y_test))
    
#     score=cross_val_score(model,X,y,cv=5)
#     print('CV Score:',np.mean(score))

In [633]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model)

# Complete training with full data

In [634]:
model =LogisticRegression()
model.fit(X,y)

In [635]:
# input split for test data
X_test = test.drop(columns=['PassengerId', 'Survived'], axis=1)

In [636]:
pred = model.predict(X_test)
pred

# SUBMIT


In [637]:
sub = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
sub.head()

In [638]:
sub.info()

In [639]:
sub['Survived'] = pred
sub['Survived'] = sub['Survived'].astype('int')

In [640]:
sub.head()

In [641]:
sub.info()

In [642]:
sub.to_csv('submission.csv', index=False)